# Cleaning_asylum.ipynb
#### This script cleans and merges relevant variables across datasets
#### flag_full = 1 considers proceedings/cases associated with full asylum applications. creates a dataset unique at the idncase level  (within proceeding, it picks the asylum application and within case, it picks the proceeding associated with the asylum application, prioritizing by date --most recent first)
#### flag_full = 0 considers a grant on any asylum case type (full, witholding, wcat) a "grant" decision on the case. 
#### Currently, it is doing cleaning and merging only for the baseline model.

In [1]:
import pandas as pd
import numpy as np
pd.set_option('precision', 5)

In [2]:
# flags to toggle between model designs
# 2 (asylum: full, any) x 2 (predictability: early, late)

flag_full = 0
flag_early = 0

if flag_full:
    filetag = 'full'
else:
    filetag = 'any'

In [3]:
path = '/data/Dropbox/Data/Asylum_Courts/raw'

## Clean court_appln.csv

relevant variables: idnProceeding, idnCase, Appl_Code

In [4]:
app = pd.read_csv(path + '/court_appln.csv', low_memory=False)

#app.count()
#app.describe()

In [5]:
# making a new variable, dec, simplifying grant decisions to DENY, GRANT, or nan
app['dec']= np.nan
app.loc[app.Appl_Dec.isin(['G','F','N','L','C']),'dec']= 1
app.loc[(app["Appl_Dec"] == 'D'),'dec'] = 0
app = app[app.dec.isin([1,0])] # only include DENY or GRANT cases

#app.count()


In [6]:
if flag_full: # full asylum cases
    # only keep applications of type ASYL. sort by date within idnproceeding
    # sorting by date--if there are multiple applications with the same decision with the same case type, 
    # take the most recent one.
    app = app[app.Appl_Code.isin(['ASYL'])]

    # sort multiple times because some need to be ascending and some descending
    app = app.sort_values(['idnProceeding','Appl_Recd_Date'],ascending=[True,False])
    #len(app)

else:
    # only keep applications of type ASYL, ASYW, WCAT. sort by Grant, then deny, then case type in order (ASYL, ASYW, WCAT)
    # then date within idnproceeding
    # sorting by date--if there are multiple applications with the same decision with the same case type, 
    # take the most recent one.
    app = app[app.Appl_Code.isin(['ASYL','ASYW', 'WCAT'])]

    # sort multiple times because some need to be ascending and some descending
    app = app.sort_values(['idnProceeding','dec','Appl_Code','Appl_Recd_Date'],ascending=[True,False,True,False])


In [7]:
# adding placeholder number to count how many applications were filed for idnProceeding
app['numAppsPerProc'] = 1
app['numAppsPerProc'] = app['numAppsPerProc'].astype('int64')

# adding additional feature based on how many applications have been filed for the same (idnCase, idnProceeding) pair
app['numAppsPerProc'] = app.groupby(['idnCase', 'idnProceeding'])['numAppsPerProc'].transform('count')

# dropping all applications with empty decisions
app = app.dropna(subset=['Appl_Dec'])

#app.describe()
app.count()

idnProceedingAppln    1155412
idnProceeding         1155412
idnCase               1155412
Appl_Code             1155412
Appl_Recd_Date        1155410
Appl_Dec              1155412
dec                   1155412
numAppsPerProc        1155412
dtype: int64

In [8]:
app = app.rename(columns={"idnCase":"idncase", "idnProceeding":"idnproceeding"})

In [9]:
# make unique--take the first application for each proceeding, when sorted in order dec (grant deny),
# case type(ASYL, ASYW, WCAT), date

app2 = app.groupby('idnproceeding', as_index=False).first()

In [10]:
app2.count()

idnproceeding         614388
idnProceedingAppln    614388
idncase               614388
Appl_Code             614388
Appl_Recd_Date        614387
Appl_Dec              614388
dec                   614388
numAppsPerProc        614388
dtype: int64

In [11]:
app2.head(10)

,idnproceeding,idnProceedingAppln,idncase,Appl_Code,Appl_Recd_Date,Appl_Dec,dec,numAppsPerProc
0,75.0,41,3328085.0,ASYL,1994-12-20 00:00:00,D,0.0,2
1,85.0,43,3328111.0,ASYL,1995-04-02 00:00:00,D,0.0,2
2,103.0,49,3328153.0,ASYL,1995-05-05 00:00:00,G,1.0,1
3,111.0,52,3328175.0,ASYL,1995-03-31 00:00:00,D,0.0,1
4,136.0,56,3327844.0,ASYL,1995-03-06 00:00:00,D,0.0,2
5,139.0,58,3327852.0,ASYL,1996-05-14 00:00:00,G,1.0,2
6,145.0,60,3327869.0,ASYL,1995-11-29 00:00:00,D,0.0,2
7,147.0,61,3327877.0,ASYL,1995-11-29 00:00:00,D,0.0,2
8,149.0,62,3327884.0,ASYL,1995-04-14 00:00:00,D,0.0,2
9,159.0,63,3327927.0,ASYL,1995-07-25 00:00:00,D,0.0,2


In [12]:
# note: idnProceedingAppln was not dropped in cleaning_full_asylum
app2 = app2.drop(columns=['idnProceedingAppln', 'Appl_Dec'])

## Clean master.csv

Relevant variables: idncase, idnproceeding, osc_date, tracid, nat

In [13]:
# load in data
master = pd.read_csv(path + '/master.csv', low_memory=False)

In [14]:
# drop empty cases and proceedings
master = master.dropna(subset= ['idncase','idnproceeding'])
#master.describe()

In [15]:
# stuff on osc_date (date charges filed or NTA)
master = master.dropna(subset=['osc_date']) # dropping empty dates

master['osc_date'] = master['osc_date'].astype('str')
master = master[master['osc_date'].apply(lambda x: len(x) == 9)] # delete dates invalid formats

master['osc_date'] = pd.to_datetime(master['osc_date'], format='%d%b%Y') # change to date format 
#master.describe()

# delete NTA dates before 1984
master = master[master.osc_date.dt.year>1983]

In [16]:
# comp date (date proceeding completed)
master = master.dropna(subset=['comp_date']) # dropping empty dates

master['comp_date'] = master['comp_date'].astype('str')
master = master[master['comp_date'].apply(lambda x: len(x) == 9)] # delete dates invalid formats

master['comp_date'] = pd.to_datetime(master['comp_date'], format='%d%b%Y') # change to date format 

#drop comp date dates before 1985
master = master[master.comp_date.dt.year>1984]


In [17]:
master.head(10)

,idncase,nat,case_type,c_asy_type,idnproceeding,base_city_code,hearing_loc_code,dec_type,dec_code,other_comp,osc_date,input_date,comp_date,attorney_flag,ij_code,tracid
11,2046920.0,MX,RMV,NaN,3200048,CHI,CHD,O,X,NaN,2004-08-06,11AUG2004,2004-08-11,NaN,RDV,31.0
12,2046921.0,MX,RMV,NaN,3200049,CHI,CHD,O,X,NaN,2004-08-06,10AUG2004,2004-08-11,NaN,RDV,31.0
13,2046922.0,MX,RMV,NaN,3200050,CHI,CHD,O,X,NaN,2004-08-09,19AUG2004,2004-08-19,NaN,JLG,29.0
14,2046923.0,PL,RMV,NaN,3200051,CHI,CHD,O,X,NaN,2004-08-09,13AUG2004,2004-08-25,1.0,CC,27.0
15,2046923.0,PL,RMV,NaN,3525150,CHI,CHD,NaN,NaN,T,2004-08-09,30MAR2005,2005-04-13,1.0,GPK,30.0
16,2046923.0,PL,RMV,NaN,3538044,CHI,CHI,O,R,NaN,2004-08-09,13APR2005,2007-06-04,1.0,CC,27.0
17,2046924.0,MX,RMV,NaN,3200052,CHI,CHD,O,X,NaN,2004-08-09,13AUG2004,2004-08-13,NaN,RDV,31.0
18,2046925.0,MX,RMV,NaN,3200053,CHI,CHD,O,X,NaN,2004-08-10,19AUG2004,2004-08-19,NaN,JLG,29.0
19,2046926.0,MX,RMV,NaN,3200054,CHI,CHD,O,X,NaN,2004-08-10,16AUG2004,2004-08-30,NaN,CMZ,32.0
20,2046927.0,MX,RMV,NaN,3200055,CHI,CHD,O,X,NaN,2004-08-12,19AUG2004,2004-08-19,NaN,JLG,29.0


In [18]:
# delete duplicates (since idnproceeding are unique, this shouldn't do anything)
master = master.drop_duplicates(subset=['idncase', 'idnproceeding'])

In [19]:
master.count()

idncase             5669748
nat                 5667915
case_type           5669747
c_asy_type          1515478
idnproceeding       5669748
base_city_code      5669737
hearing_loc_code    5669646
dec_type            4534187
dec_code            4379061
other_comp          1290572
osc_date            5669748
input_date          5667795
comp_date           5669748
attorney_flag       2743993
ij_code             5637391
tracid              5360370
dtype: int64

In [20]:
master['idnproceeding'] = master['idnproceeding'].astype('float64')


In [21]:
#replace nan attorney flags with 0.
master.loc[pd.isnull(master.attorney_flag),'attorney_flag']=0


## Merge master and court_appln

In [22]:
merged = pd.merge(app2, master, on=['idnproceeding','idncase'])

In [23]:
merged.count()

idnproceeding       608560
idncase             608560
Appl_Code           608560
Appl_Recd_Date      608559
dec                 608560
numAppsPerProc      608560
nat                 608403
case_type           608560
c_asy_type          595298
base_city_code      608558
hearing_loc_code    608557
dec_type            605465
dec_code            605170
other_comp            3353
osc_date            608560
input_date          608547
comp_date           608560
attorney_flag       608560
ij_code             607676
tracid              594055
dtype: int64

In [24]:
merged.head(10)

,idnproceeding,idncase,Appl_Code,Appl_Recd_Date,dec,numAppsPerProc,nat,case_type,c_asy_type,base_city_code,hearing_loc_code,dec_type,dec_code,other_comp,osc_date,input_date,comp_date,attorney_flag,ij_code,tracid
0,75.0,3328085.0,ASYL,1994-12-20 00:00:00,0.0,2,HO,DEP,E,PIS,PIS,O,V,NaN,1994-11-03,10NOV1994,1995-03-10,1.0,CAL,NaN
1,85.0,3328111.0,ASYL,1995-04-02 00:00:00,0.0,2,HO,DEP,E,HOU,HOU,O,V,NaN,1994-11-04,14DEC1994,1997-06-16,1.0,WKZ,71.0
2,103.0,3328153.0,ASYL,1995-05-05 00:00:00,1.0,1,GT,DEP,E,NYC,NYC,W,D,NaN,1994-11-05,20DEC1994,1995-08-08,1.0,JSC,139.0
3,111.0,3328175.0,ASYL,1995-03-31 00:00:00,0.0,1,ES,DEP,E,HOU,HOU,O,V,NaN,1994-11-05,06DEC1994,1995-08-15,1.0,CMR,70.0
4,136.0,3327844.0,ASYL,1995-03-06 00:00:00,0.0,2,HO,DEP,E,HLG,HLG,O,D,NaN,1995-02-06,06MAR1995,1995-04-06,1.0,JZ,50.0
5,139.0,3327852.0,ASYL,1996-05-14 00:00:00,1.0,2,CU,DEP,E,MIA,MIA,O,R,NaN,1995-02-06,05OCT1995,1996-05-14,1.0,RAJ,126.0
6,145.0,3327869.0,ASYL,1995-11-29 00:00:00,0.0,2,NU,DEP,E,MIA,MIA,O,V,NaN,1995-02-10,23MAY1995,1996-01-04,1.0,WKZ,71.0
7,147.0,3327877.0,ASYL,1995-11-29 00:00:00,0.0,2,NU,DEP,E,MIA,MIA,O,V,NaN,1995-02-10,12APR1995,1996-01-04,1.0,WKZ,71.0
8,149.0,3327884.0,ASYL,1995-04-14 00:00:00,0.0,2,NU,DEP,E,PIS,PIS,O,V,NaN,1995-02-12,22FEB1995,1995-06-23,0.0,MB,61.0
9,159.0,3327927.0,ASYL,1995-07-25 00:00:00,0.0,2,NU,DEP,E,HOU,HOU,O,V,NaN,1995-02-17,15MAR1995,1995-09-05,1.0,CMR,70.0


In [25]:
# drop nan tracids 
merged = merged.dropna(subset=['tracid'])

In [26]:
# drop all cases where judge has fewer than 100 cases--same as in gambler's fallacy paper
tracid_100 = merged.groupby('tracid').idnproceeding.count() >= 100 #bool indicating whether judge has at least 100 cases
tracid_100 = tracid_100.index.values[tracid_100]#indices of judges with at least 100 cases
merged2 = merged.loc[merged.tracid.isin(tracid_100)]
#merged2.count()

In [27]:
merged2.head(5)

,idnproceeding,idncase,Appl_Code,Appl_Recd_Date,dec,numAppsPerProc,nat,case_type,c_asy_type,base_city_code,hearing_loc_code,dec_type,dec_code,other_comp,osc_date,input_date,comp_date,attorney_flag,ij_code,tracid
1,85.0,3328111.0,ASYL,1995-04-02 00:00:00,0.0,2,HO,DEP,E,HOU,HOU,O,V,NaN,1994-11-04,14DEC1994,1997-06-16,1.0,WKZ,71.0
2,103.0,3328153.0,ASYL,1995-05-05 00:00:00,1.0,1,GT,DEP,E,NYC,NYC,W,D,NaN,1994-11-05,20DEC1994,1995-08-08,1.0,JSC,139.0
3,111.0,3328175.0,ASYL,1995-03-31 00:00:00,0.0,1,ES,DEP,E,HOU,HOU,O,V,NaN,1994-11-05,06DEC1994,1995-08-15,1.0,CMR,70.0
4,136.0,3327844.0,ASYL,1995-03-06 00:00:00,0.0,2,HO,DEP,E,HLG,HLG,O,D,NaN,1995-02-06,06MAR1995,1995-04-06,1.0,JZ,50.0
5,139.0,3327852.0,ASYL,1996-05-14 00:00:00,1.0,2,CU,DEP,E,MIA,MIA,O,R,NaN,1995-02-06,05OCT1995,1996-05-14,1.0,RAJ,126.0


## Load and merge tblLookupHloc and tblLookupBaseCity

In [28]:
# mapping hearing_loc_code
hearingloc_map = pd.read_csv(path + '/tblLookupHloc.csv', header=None)

In [29]:
hearingloc_map.head(5)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,2,AAP,ATLANTA AIRPORT,** INACTIVE HEARING LOCATION **,"CONCOURSE E., ROOM A",ATLANTA,GA,30320,9999999999,ATL,...,2,0,0,0,30,0,0,11,False,False
1,288,ABQ,REGIONAL CORRECTIONS CENTER,REGIONAL CORRECTIONS CENTER,415 ROMA NW,ALBUQUERQUE,NM,87102,5052426177,ELP,...,2,0,0,0,30,0,0,10,False,True
2,3,ADC,DHS-Litigation Unit/Oakdale,IMMIGRATION COURT,1900 East Whatley Road,OAKDALE,LA,71463,318335365,OAK,...,0,0,0,0,30,0,0,5,False,True
3,326,ADE,ADELANTO DETENTION FACILITY EAST,IMMIGRATION COURT,10400 RANCHO ROAD,ADELANTO,CA,923012237,2138942811,ADL,...,2,0,0,0,30,0,0,9,True,True
4,341,ADL,ADELANTO,IMMIGRATION COURT,"10250 RANCHO RD., SUITE 201A",ADELANTO,CA,92301,1111111111,ADL,...,2,0,0,0,30,0,0,9,False,False


In [30]:
hearingloc_map = hearingloc_map.rename(columns={1:'hearing_loc_code', 5:'hearing_city'})
merged2['hearing_loc_code'] = merged2['hearing_loc_code'].astype('str').str.strip()
hearingloc_map['hearing_loc_code'] = hearingloc_map['hearing_loc_code'].astype('str').str.strip()

/usr/local/anaconda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [31]:
hearingloc_map = hearingloc_map[['hearing_loc_code', 'hearing_city']]

In [32]:
merged2 = pd.merge(merged2, hearingloc_map, on=['hearing_loc_code'], how='left')

In [33]:
#merged2.loc[pd.isnull(merged2.hearing_city),'hearing_loc_code'] = 'UNKNOWN'

In [34]:
merged2['hearing_city'] = merged2['hearing_city'].fillna('UNKNOWN')
merged2['hearing_loc_code'] = merged2['hearing_loc_code'].fillna('UNKNOWN')

In [35]:
basecity_map = pd.read_csv(path + '/tblLookupBaseCity.csv', header=None)
basecity_map = basecity_map.rename(columns={1:'base_city_code', 5:'base_city'})
basecity_map = basecity_map[['base_city_code','base_city']]

In [36]:
merged2['base_city_code'] = merged2['base_city_code'].astype('str').str.strip()
basecity_map['base_city_code'] = basecity_map['base_city_code'].astype('str').str.strip()

In [37]:
merged2 = pd.merge(merged2, basecity_map, on=['base_city_code'], how='left')

In [38]:
merged2['base_city'] = merged2['base_city'].fillna('UNKNOWN')
merged2['base_city_code'] = merged2['base_city_code'].fillna('UNKNOWN')

In [39]:
merged2.count()

idnproceeding       593112
idncase             593112
Appl_Code           593112
Appl_Recd_Date      593111
dec                 593112
numAppsPerProc      593112
nat                 592964
case_type           593112
c_asy_type          580139
base_city_code      593112
hearing_loc_code    593112
dec_type            590367
dec_code            590075
other_comp            3000
osc_date            593112
input_date          593100
comp_date           593112
attorney_flag       593112
ij_code             593112
tracid              593112
hearing_city        593112
base_city           593112
dtype: int64

## feature cleanup on the variable containing merged information

In [40]:
# adding additional feature based on how many asylum proceedings have been filed for the same (idnCase) 
merged2['numProcPerCase'] = 1
merged2['numProcPerCase'] = merged2['numProcPerCase'].astype('int64')
merged2['numProcPerCase'] = merged2.groupby(['idncase'])['numProcPerCase'].transform('count')

# make unique at idncase level, sorting with the same logic as used to sort applications
if flag_full:
    merged_case = merged2.sort_values(['idncase','Appl_Recd_Date'],ascending=[True,False])
else: 
    #counting case as a grant if ANY proceeding was grant
    merged_case = merged2.sort_values(['idncase','dec','Appl_Code','Appl_Recd_Date'],ascending=[True,False,True,False])
    
merged_case = merged_case.groupby('idncase',as_index=False ).first()

In [41]:
merged_case.groupby('dec').count()

,idncase,idnproceeding,Appl_Code,Appl_Recd_Date,numAppsPerProc,nat,case_type,c_asy_type,base_city_code,hearing_loc_code,...,other_comp,osc_date,input_date,comp_date,attorney_flag,ij_code,tracid,hearing_city,base_city,numProcPerCase
dec,,,,,,,,,,,,,,,,,,,,,
0.0,339046,339046,339046,339046,339046,338928,339046,328644,339046,339046,...,1687,339046,339036,339046,339046,339046,339046,339046,339046,339046
1.0,226110,226110,226110,226109,226110,226084,226110,223804,226110,226110,...,1309,226110,226108,226110,226110,226110,226110,226110,226110,226110


In [42]:
# get rid of merged_cases where other_comp is not null. other_comp indicates that the proceeding ended for a reason other than 
# a judge's decision, suggesting no decision was actually made. this is less than 1% of cases once we have already filtered
# out applications where the decision is not grant or deny and matched them to proceedings.
merged_case = merged_case[pd.isnull(merged_case.other_comp)]

# get rid of cases that don't have c_asy type (about 2% of cases--higher proportion than full_asylum version...)
merged_case = merged_case[~pd.isnull(merged_case.c_asy_type)] 

# change values of c_asy_type to be more clear
merged_case.loc[merged_case.c_asy_type=='I','c_asy_type'] = 'aff'
merged_case.loc[merged_case.c_asy_type=='E','c_asy_type'] = 'def'

In [43]:
# drop variables that definitely won't be used as features (or won't be used to track where the data came from)

merged_case = merged_case.drop(columns=['Appl_Code','Appl_Recd_Date','dec_type','other_comp','input_date','ij_code','dec_code'])

In [44]:
# change ?? to unknwon for  159 cases with unknown nationalities
merged_case.loc[(merged_case.nat=='??'),'nat'] = 'UNKNOWN'

# mark na nats as unknown
merged_case.loc[pd.isnull(merged_case.nat),'nat'] = 'UNKNOWN'
# load nationality lookup table
nat_lut =  pd.read_csv(path+ '/tblLookupNationality.csv',header=None)

# mark 4 observations where the nationality code is not in the lookup table as unknown
merged_case.loc[~merged_case.nat.isin(nat_lut[1]),'nat'] = 'UNKNOWN'

# mark as unknown 2 observations with nationality code XX whic the LUT says corresponds 
# to "BE REMOVED FROM THE UNITED STATES"
merged_case.loc[(merged_case.nat=="XX"),'nat'] = 'UNKNOWN'

# examine counts for different nationalities:
nat_numbers = merged_case.groupby('nat',as_index=False).idncase.count().sort_values('idncase')

# some nationalities have only 1 or 2 observations. drop any with less than 10 observations.
nat_10 = nat_numbers.loc[nat_numbers['idncase']>9,'nat']
merged_case = merged_case[merged_case.nat.isin(nat_10)]


In [45]:
# remove cities with less than 10 obs (only removes 2 cases)

city_numbers = merged_case.groupby('base_city_code',as_index=False).idncase.count().sort_values('idncase')
cities_10 = city_numbers.loc[city_numbers['idncase']>9,'base_city_code']

merged_case = merged_case[merged_case.base_city_code.isin(cities_10)]


In [46]:
# hearing loc--is this "court"? some of them are prisons/detention centers/airports. many "courts" have fewer than 10 obs
# drop thewse (less than 1% of proceedings)

court_numbers = merged_case.groupby('hearing_loc_code',as_index=False).idncase.count().sort_values('idncase')

courts_10 = court_numbers.loc[court_numbers['idncase']>9,'hearing_loc_code']
merged_case = merged_case[merged_case.hearing_loc_code.isin(courts_10)]


In [47]:
merged_case.groupby('dec').count()

,idncase,idnproceeding,numAppsPerProc,nat,case_type,c_asy_type,base_city_code,hearing_loc_code,osc_date,comp_date,attorney_flag,tracid,hearing_city,base_city,numProcPerCase
dec,,,,,,,,,,,,,,,
0.0,326795,326795,326795,326795,326795,326795,326795,326795,326795,326795,326795,326795,326795,326795,326795
1.0,222440,222440,222440,222440,222440,222440,222440,222440,222440,222440,222440,222440,222440,222440,222440


In [48]:
# save data
merged_case.to_csv('/data/WorkData/spatialtemporal/merged_master_app_' + filetag + '.csv',index=False)